<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Run-Test-Simulation" data-toc-modified-id="Run-Test-Simulation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Run Test Simulation</a></span><ul class="toc-item"><li><span><a href="#Performances-Profiling" data-toc-modified-id="Performances-Profiling-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Performances Profiling</a></span></li></ul></li><li><span><a href="#Parameters-Grid-Search" data-toc-modified-id="Parameters-Grid-Search-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parameters Grid Search</a></span></li></ul></div>

# Intro 
This notebook explores slime mold simulation and visualization. For an introduction to the phenomenon and method see [this sagejenson post](https://sagejenson.com/physarum)

In [ ]:
import numpy as np
import cupy as cp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import starmap, product
import tqdm

import os
import sys
from pathlib import Path

%matplotlib inline

%load_ext autoreload
%autoreload 2

import Physarum as physarum
from Physarum import Physarum

from ds_utils.video_utils import generate_video, imageio_generate_video

# Run Test Simulation

In [ ]:
width = 400
height = 400
system_shape = (width, height)

In [ ]:
init_fun_perlin=lambda shape: physarum.get_perlin_init(shape=shape, n=int(30e4), scale=380)
init_fun_circle=lambda shape: physarum.get_filled_circle_init(n=int(10e4), center=(shape[0]//2,shape[1]//2), 
                                             radius=100)

In [ ]:
def combined_init(shape):
    pop_01 = physarum.get_filled_circle_init(n=int(10e4), center=(shape[0]//2,shape[1]//2), radius=100)
    pop_02 = physarum.get_perlin_init(shape=shape, n=int(30e4), scale=80)
    return np.concatenate([pop_01, pop_02])

In [ ]:
species_a = Physarum(shape=system_shape, horizon_walk=1, horizon_sense=9,
                    theta_walk=15, theta_sense=10., walk_range=1.,
                    social_behaviour=0, trace_strength=1,
                    init_fun=combined_init)


species_b = Physarum(shape=system_shape, horizon_walk=1,horizon_sense=9,
                    theta_walk=15, theta_sense=10., walk_range=1.2,
                    social_behaviour=-16,trace_strength=1,
                    init_fun=init_fun_circle)

In [ ]:
simulation_steps = 10
images = physarum.run_physarum_simulation(populations=[species_a], steps=simulation_steps)

In [ ]:
out_path = Path.home() / 'Documents/graphics/generative_art_output/physarum/test_01'
out_path.mkdir(exist_ok=True, parents=True)

In [ ]:
imageio_generate_video(str(out_path/"test_02.mp4"), images, fps=20, format="mp4", loop=False)

In [ ]:
generate_video(str(out_path/"tmp.mp4"), (width, height),
               frame_gen_fun = lambda i: np.array(images[i])[:,:,:3],
               nb_frames = len(images))

## Performances Profiling

In [ ]:
%%prun -s cumulative -l 30 -r
# We profile the cell, sort the report by "cumulative
# time", limit it to 30 lines

simulation_steps = 50
images = physarum.run_physarum_simulation(populations=[species_b], steps=simulation_steps)

# Parameters Grid Search 

In [ ]:
def named_configs(items):
    Config = namedtuple('Config', items.keys())
    return starmap(Config, product(*items.values()))

In [ ]:
out_path = Path.home() /  'Documents/graphics/generative_art_output/physarum/grid_search'

In [ ]:
NB_VALS = 4
grid_search_params = {
    'horizon_walk': np.linspace(1., 10., NB_VALS).round(2), # higher more spread, chaos
    'horizon_sense': np.linspace(3., 30., NB_VALS).round(2),
    'theta_sense': np.linspace(6., 45., NB_VALS).round(2), # the smaller, the more narrow paths they create
    'theta_walk': [0.],#np.linspace(15, 30., NB_VALS).round(2), # should be close to theta_sense, if way bigger, they disappear or constrain to concentrated areas
    'walk_range': [1.],
    'social_behaviour': [0],
    'trace_strength': [1],
    'decay': [0.8], #np.linspace(.6, .9, NB_VALS)
}
configs = list(named_configs(grid_search_params))

In [ ]:
system_shape = (500, 500)
render_dir = out_path / f"gs_fix_perlin_{system_shape[0]}"
nb_frames = 340

render_dir.mkdir(exist_ok=False, parents=True)

init_setup = physarum.get_perlin_init(shape=system_shape, n=int(60e4), scale=150)
#init_setup = physarum.get_filled_circle_init(n=int(50e4), center=(system_shape[0]//2,system_shape[1]//2), radius=100)
def combined_init(shape):
    pop_01 = physarum.get_filled_circle_init(n=int(80e4), center=(shape[0]//2,shape[1]//2), radius=150)
    pop_02 = physarum.get_perlin_init(shape=shape, n=int(10e4), scale=200)
    #pop_01 = physarum.get_gaussian_gradient(n=int(10e4), center=(system_shape[0]//2,system_shape[1]//2), sigma=200)
    #pop_02 = physarum.get_circle_init(n=int(5e4), center=(shape[0]//2,shape[1]//2), radius=100, width=30)
    return cp.concatenate([pop_01, pop_02])
#init_setup = combined_init(system_shape)
#init_setup = physarum.get_gaussian_gradient(n=int(10e4), center=(system_shape[0]//2,system_shape[1]//2), sigma=200)

with open(str(render_dir / "logs.txt"), 'w+') as f:
    for run, config in tqdm.notebook.tqdm(enumerate(configs)):
        config = config._replace(theta_walk = config.theta_sense-5)
        
        f.write(str(config)+"\n")
        SYSTEM_CONFIG = config._asdict()

        system = Physarum(shape=system_shape, 
                          horizon_walk=config.horizon_walk,
                          horizon_sense=config.horizon_sense,
                          theta_walk=config.theta_walk,
                          theta_sense=config.theta_sense,
                          walk_range=config.walk_range,
                          social_behaviour=config.social_behaviour,
                          trace_strength=config.trace_strength,
                          init_fun=lambda shape: init_setup)
        
        images = physarum.run_physarum_simulation(populations=[system], diffusion='median',
                                                  steps=nb_frames, decay=config.decay)
        
        # images fix for rendering
        fix_images = np.sqrt(images + 0.1) - np.sqrt(0.1)
        fix_images = np.log(fix_images + 1)
        fix_images = np.uint8((fix_images / np.max(fix_images))*255)

        out = render_dir / f'run_{run}.mp4'
        #imageio_generate_video(str(out), images, fps=30, format="mp4", loop=False)
        
        generate_video(str(out), (system_shape[1], system_shape[0]),
               frame_gen_fun = lambda i: fix_images[i],
               nb_frames = len(fix_images), is_color=False, disable_tqdm=True)